In [4]:
import pandas as pd
import numpy as np

### Сравните две программы по успеваемости

   


In [125]:
columns = ["id","program_name","performance","is_ege","age"]
sample_data = pd.read_csv('hw1.csv', sep=';', names=columns, skiprows=1, decimal=',')
# sample_data['is_ege'] = sample_data.is_ege.map({'yes':1, 'no':0})
# columns_types = {'id': 'int64', 'program_name': 'object', 'performance': 'float64', 'is_ege': 'bool', 'age': 'int64'}
# sample_data = sample_data.astype(columns_types)

sample_data

,id,program_name,performance,is_ege,age
0,1,math&phys,0.606203,no,17
1,2,math&phys,0.648850,yes,18
2,3,math&phys,0.729141,no,17
3,4,math&phys,0.863283,yes,17
4,5,math&phys,0.580673,no,17
5,6,math&phys,0.859356,yes,18
6,7,math&phys,0.877870,no,18
7,8,math&phys,0.764319,yes,17
8,9,math&phys,0.751646,no,18
9,10,math&phys,0.524715,no,18


In [126]:
sample_data.dtypes

id                int64
program_name     object
performance     float64
is_ege           object
age               int64
dtype: object

- Посчитайте: среднее, дисперсию и стандартное отклонение успеваемости
 

In [128]:
print('Успеваемость')
sample_stats_by_program = sample_data.groupby(by='program_name', as_index=False).performance.agg([
    "count", # количество записей
    "mean",  # среднее
    "var",   # дисперсия
    "std",   # стандартное отклонение
    "sem",   # стандартная ошибка
]) 
sample_stats_by_program

Успеваемость


,program_name,count,mean,var,std,sem
0,math&phys,10,0.720606,0.015942,0.126262,0.039928
1,russ&lit,25,0.797763,0.012154,0.110245,0.022049


In [129]:


#перепроверка стандартной ошибки
print(f'math&phys -- {0.126262/np.sqrt(10)}')
print(f'russ&lit -- {0.110245/np.sqrt(25)}')


math&phys -- 0.03992755019281799
russ&lit -- 0.022049


- Посчитайте: доверительный интервал 90%, 95% и 99%


In [130]:

ci_columns = ['confidence_level', 'ci_lower_math', 'ci_upper_math',  'ci_lower_russ', 'ci_upper_russ']

def get_ci(confidence_level, mean, sem):
    z_scores = {0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    ci_upper = mean + z_scores[confidence_level] * sem
    ci_lower = mean - z_scores[confidence_level] * sem

    return ci_lower, ci_upper

data = []

for cl in [0.9, 0.95, 0.99]:
    mean_math = sample_stats_by_program[sample_stats_by_program["program_name"] == 'math&phys']['mean'].values[0]
    sem_math = sample_stats_by_program[sample_stats_by_program["program_name"] == 'math&phys']['sem'].values[0]
    mean_russ = sample_stats_by_program[sample_stats_by_program["program_name"] == 'russ&lit']['mean'].values[0]
    sem_russ = sample_stats_by_program[sample_stats_by_program["program_name"] == 'russ&lit']['sem'].values[0]
    ci_lower_math, ci_upper_math = get_ci(cl, mean_math, sem_math)
    ci_lower_russ, ci_upper_russ = get_ci(cl, mean_russ, sem_russ)

    data.append([cl, ci_lower_math, ci_upper_math, ci_lower_russ, ci_upper_russ])

    
conf_intervals = pd.DataFrame(data, columns=ci_columns)


conf_intervals



,confidence_level,ci_lower_math,ci_upper_math,ci_lower_russ,ci_upper_russ
0,0.90,0.654725,0.786486,0.761382,0.834144
1,0.95,0.642347,0.798864,0.754547,0.840980
2,0.99,0.617592,0.823619,0.740877,0.854650


- Посчитайте: входят ли `program_name` в один доверительный интервал по успеваемости (пересекаются ли доверительные интервалы между группами)


In [131]:
conf_intervals['have_overlaping_intervals'] = (
    ((conf_intervals['ci_lower_math'] >= conf_intervals['ci_lower_russ']) & (conf_intervals['ci_lower_math'] <= conf_intervals['ci_upper_russ']))
    |
    ((conf_intervals['ci_lower_russ'] >= conf_intervals['ci_lower_math']) & (conf_intervals['ci_lower_russ'] <= conf_intervals['ci_upper_math']))
    |
    ((conf_intervals['ci_upper_math'] >= conf_intervals['ci_lower_russ']) & (conf_intervals['ci_upper_math'] <= conf_intervals['ci_upper_russ']))
    |
    ((conf_intervals['ci_upper_russ'] >= conf_intervals['ci_lower_math']) & (conf_intervals['ci_lower_russ'] <= conf_intervals['ci_upper_math']))
)

print("Да program_name входят в один доверительный интервали по успеваемости")
conf_intervals


Да program_name входят в один доверительный интервали по успеваемости


,confidence_level,ci_lower_math,ci_upper_math,ci_lower_russ,ci_upper_russ,have_overlaping_intervals
0,0.90,0.654725,0.786486,0.761382,0.834144,True
1,0.95,0.642347,0.798864,0.754547,0.840980,True
2,0.99,0.617592,0.823619,0.740877,0.854650,True


- Посчитайте и напишите вывод: у какой группы `is_ege` разброс по средней успеваемости больше


In [134]:
print('Разброс(?). Стандартное отклонение ниже у тех, кто сдает ЕГЭ. Значит "разброс" больше у тех, кто его не сдает')
sample_data.groupby(by='is_ege', as_index=False).performance.agg([
    "count", # количество записей
    "mean",  # среднее
    "var",   # дисперсия
    "std",   # стандартное отклонение
    "sem",   # стандартная ошибка
]) 

Разброс(?). Стандартное отклонение ниже у тех, кто сдает ЕГЭ. Значит "разброс" больше у тех, кто его не сдает


,is_ege,count,mean,var,std,sem
0,no,25,0.768338,0.016618,0.128912,0.025782
1,yes,10,0.794170,0.008233,0.090735,0.028693


## Предположите, какие выводы можно сделать на основе анализа

- Русский почти в два раза популярнее математики в нашем сервисе
- 25 из 35 учеников не сдают ЕГЭ в этом году. Значит ученики в основном начинают готовиться заранее или нам не доверяют подготовку в финальные месяцы перед эказаменом
- У тех кто сдает ЕГЭ в этом году успеваемость в среднем выше, стоит проверить значимо ли это отличие
- В среднем успеваемость по русскому лучше, чем по математике, но отличие не стат значимо

## Вопросы

Правильно ли смотреть пересеклись ли дов интревалы, чтобы сделать вывод о значимости изменения